In [99]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

objects = {
    'ode_textbook': {'type': 'object', 'dimension': [8.5, 10.25, 1.25]},
    'perl_cookbook': {'type': 'object', 'dimension': [7, 9.25, 1.5]},
    'box1': {'type': 'box', 'dimension': [18, 13.75, 5], 'description': 'prafull sharma'},
    'box2': {'type': 'box', 'dimension': [8.75, 12, 5.5], 'description': 'daniel xu'},
    'oat_creamer': {'type': 'object', 'dimension': [8, 3, 3]},
    'box3': {'type': 'box', 'dimension': [17.5, 13.5, 7.5], 'description': 'lays snack'},
    'pillow': {'type': 'object', 'dimension': [14, 14, 3]},
    'straws' : {'type': 'object', 'dimension': [10, 10, 3.5]},
    'box4': {'type': 'box', 'dimension': [9.5, 12.25, 7], 'description': 'pz'},
    'box5': {'type': 'box', 'dimension': [8.75, 11.75, 5.25], 'description': 'lance ying, 10/7'},
    'box6': {'type': 'box', 'dimension': [8, 12, 10.5], 'description': 'marie mize'},
    'box7': {'type': 'box', 'dimension': [14.25, 14.25, 5.25], 'description': 'rice crispy treats'},
    'hack_water_bottle': {'type': 'object', 'dimension': [10, 3, 3]},
    'main_street_cafe_mug': {'type': 'object', 'dimension': [10, 3.5, 3.5]},
    'light_blue_mug': {'type': 'object', 'dimension': [4.5, 4, 4]},
    'yoga_mat': {'type': 'object', 'dimension': [18, 4.5, 4.5]},
    'laptop_with_box': {'type': 'object', 'dimension': [14, 10.25, 4]},
}


In [86]:
def get_volume(obj):
    return obj.dimensions[0] * obj.dimensions[1] * obj.dimensions[2]

def generate_object(obj):
    return Object(obj, "none", objects[obj]['dimension'])

get_volume(generate_object('laptop_with_box')) + get_volume(generate_object('yoga_mat'))



1060.0

In [93]:
get_volume(generate_object('box1')) 

1237.5

In [92]:
get_volume(generate_object('box7')) 

643.453125

In [89]:
get_volume(generate_object('pillow')) 

867

In [101]:
class Object:
    def __init__(self, name, material, dimensions):
        self.name = name
        self.dimensions = dimensions

    def get_name(self):
        return self.name

    def get_dimensions(self):
        return self.dimensions

class InferenceModel:
    def __init__(self, objects, boxes):
        self.objects = objects
        self.boxes = boxes
        self.partitions = []
        self.partition_weights = []
        self.generate_partitions(objects, len(objects), len(boxes))

        # print(self.partition_weights)


    def generate_partitions(self, objects, n, k):
        self.partKSubsets(objects, n, k)

        unordered_partitions = self.partitions.copy()
        self.partitions = []

        for partition in unordered_partitions:
            for perm in permutations(partition):
                self.partitions.append(perm)

        self.partition_weights = [1/len(self.partitions) for partition in self.partitions]

    def check_dimension(self, partition, boxes):
        for i in range(len(boxes)):

            vol = 0
            box = boxes[i]
            box_vol = box.dimensions[0] * box.dimensions[1] * box.dimensions[2]

            for obj in partition[i]:
                if max(obj.dimensions) > max(box.dimensions) or min(obj.dimensions) > min(box.dimensions):
                    return False
                
                vol += obj.dimensions[0] * obj.dimensions[1] * obj.dimensions[2]

            if vol > box_vol:
                return False
        
        return True

    def PartitionSub(self, arr, i, N, K, nos, v):
   
        # If count of elements in K subsets 
        # are greater than or equal to N 
        if (i >= N):
        
            # If count of subsets 
            # formed is equal to K 
            if (nos == K):
                self.partitions.append([i.copy() for i in v])
            return
        for j in range(K):
        
            # If any subset is occupied, 
            # then push the element 
            # in that first 
            if (len(v[j]) > 0):
                v[j].append(arr[i]) 
    
                # Recursively do the same 
                # for remaining elements 
                self.PartitionSub(arr, i + 1, N, K, nos, v)
                
                # Backtrack 
                v[j].remove(v[j][len(v[j]) - 1])

            else:
                v[j].append(arr[i]) 
                self.PartitionSub(arr, i + 1, N, K, nos + 1, v) 
                v[j].remove(v[j][len(v[j]) - 1]) 
    
                break

    def partKSubsets(self, arr, N, K):
   
        v = [[] for i in range(K)] 
        if (K == 0 or K > N):
            print("Not Possible", end = "") 
        else:
            # print("The Subset Combinations are: ") 
            self.PartitionSub(arr, 0, N, K, 0, v)

    def get_partitions(self):

        names = []

        for partition in self.partitions:
            name = []
            for i in range(len(partition)):
                name.append([obj.get_name() for obj in partition[i]])

            names.append(name)
        return names
    
    def filter_by_dimension(self):
        for i, partition in enumerate(self.partitions):
            if not self.check_dimension(partition, self.boxes):
                self.partition_weights[i] = 0

        # print(self.partition_weights)
        self.partition_weights = [weight/sum(self.partition_weights) for weight in self.partition_weights]
        # print(self.partition_weights)
    def query_item_prob(self, item):
        prob = [0] * len(self.boxes)
        for i, partition in enumerate(self.partitions):
            for j, box in enumerate(partition):
                for obj in box:
                    if item == obj.get_name():
                        prob[j] += self.partition_weights[i]
        return prob


    def get_objects(self):
        return self.objects

    def get_boxes(self):
        return self.boxes

 
# Driver Code 
if __name__ == '__main__':

    box_namelist = ["box1", "box7"]
    items_namelist = ["yoga_mat","laptop_with_box","pillow"]

    box_list = []
    items_list = []

    for box in box_namelist:
        box_list.append(Object(box, 'none', objects[box]['dimension']))

    for item in items_namelist:
        items_list.append(Object(item, 'none', objects[item]['dimension']))

    model = InferenceModel(items_list, box_list)

    model.filter_by_dimension()

    print(model.query_item_prob("yoga_mat"))


[1.0, 0.0]


In [23]:
items_list

['yoga_mat', 'laptop_with_box', 'pillow']